# Subagents を利用したマルチエージェント構成の例

下記のブログ記事で解説している例です。

- [【マルチエージェント徹底入門】Agent Development Kit ではじめるマルチエージェント開発](https://zenn.dev/google_cloud_jp/articles/c5fa102f468cdf)

In [1]:
import copy, json, os, re, uuid
import vertexai
from google.genai.types import Part, UserContent, ModelContent
from google.adk.agents.llm_agent import LlmAgent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools.agent_tool import AgentTool

[PROJECT_ID] = !gcloud config list --format 'value(core.project)'
LOCATION = 'us-central1'

vertexai.init(project=PROJECT_ID, location=LOCATION)

os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID
os.environ['GOOGLE_CLOUD_LOCATION'] = LOCATION
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'True'

In [2]:
class LocalApp:
    def __init__(self, agent):
        self._agent = agent
        self._user_id = 'local_app'
        self._runner = Runner(
            app_name=self._agent.name,
            agent=self._agent,
            artifact_service=InMemoryArtifactService(),
            session_service=InMemorySessionService(),
            memory_service=InMemoryMemoryService(),
        )
        self._session = self._runner.session_service.create_session(
            app_name=self._agent.name,
            user_id=self._user_id,
            state={},
            session_id=uuid.uuid4().hex,
        )
        
    async def _stream(self, query):
        content = UserContent(parts=[Part.from_text(text=query)])
        async_events = self._runner.run_async(
            user_id=self._user_id,
            session_id=self._session.id,
            new_message=content,
        )
        result = []
        agent_name = None
        async for event in async_events:
            if DEBUG:
                print(f'----\n{event}\n----')
            if (event.content and event.content.parts):
                response = ''
                for p in event.content.parts:
                    if p.text:
                        response += f'[{event.author}]\n\n{p.text}\n'
                if response:
                    #### Temporary fix for wrong agent routing message
                    pattern = 'transfer_to_agent\(agent_name=["\']([^"]+)["\']\)'
                    matched = re.search(pattern, response)
                    if (not agent_name) and matched:
                        agent_name = matched.group(1)
                    else:
                        print(response)
                        result.append(response)
                    ####
        return result, agent_name

    async def stream(self, query):
        result, agent_name = await self._stream(query)
        #### Temporary fix for wrong agent routing message
        if agent_name:
            if DEBUG:
                print(f'----\nForce transferring to {agent_name}\n----')
            result, _ = await self._stream(f'Please transfer to {agent_name}')
        ####
        return result

In [3]:
shopping_mall_info = '''
* 立地と外観:
  - 新宿駅南口から徒歩5分。賑やかな駅周辺から少し離れ、落ち着いた雰囲気のエリアに位置しています。
  - 緑豊かなオープンテラスが特徴的で、都会の中にありながらも自然を感じられる空間を提供しています。
  - 夜になると、間接照明が灯り、ロマンチックな雰囲気に包まれます。

* イベント:
  - 週末には、ジャズライブやアコースティックライブなどの音楽イベントがテラスで開催され、夜の雰囲気を盛り上げます。
  - 季節ごとのイルミネーションが美しく、訪れる人の目を楽しませます。
  - 地域住民向けのワークショップやマルシェなども開催され、地域との交流を深めています。

* テナント:
  - 個性的なセレクトショップ: 大手チェーン店だけでなく、オーナーのこだわりが詰まった隠れ家のようなセレクトショップが点在しています。
  - こだわりのレストランやカフェ: 「夜の帳」のように、落ち着いた雰囲気で質の高い食事や飲み物を楽しめるお店が集まっています。テラス席があるお店も多く、開放的な空間で食事を楽しめます。
  - 上質なライフスタイル雑貨店: 日常を豊かにする、デザイン性の高い雑貨や家具、オーガニックコスメなどを扱うお店があります。
  - アートギャラリーやミニシアター: 感性を刺激するアートや映画に触れることができるスペースがあります。
'''

coffee_shop_info = '''
* 店名: 夜の帳（よるのとばり）

* コンセプト: 一日の終わりに、静かに心と体を休ませる隠れ家のような喫茶店。落ち着いた照明と、心地よい音楽が流れる空間で、こだわりのコーヒーや軽食、デザートを提供します。

* 立地と外観:
  - 新宿スターライトテラス内の、メインフロアから少し奥まった静かな一角。3階の吹き抜けに面した見晴らしの良い場所
  - オレンジや琥珀色の暖色系間接照明が、店内から優しく漏れる。控えめな光で照らされた、筆記体のような上品な看板。

* メニュー:
  ** こだわりの珈琲:
    - 夜の帳ブレンド: 深煎りでコクがあり、ほんのりビターな大人の味わい。疲れた心に染み渡ります。
    - 月光の浅煎り: フルーティーな香りが特徴の、すっきりとした味わい。リフレッシュしたい時に。
    - カフェ・オ・レ: 丁寧に淹れたブレンドコーヒーと、温かいミルクの優しいハーモニー。
    - 水出し珈琲: じっくりと時間をかけて抽出した、まろやかで雑味のないアイスコーヒー。

  ** 軽食:
    - 厚切りトーストのたまごサンド: ふわふわの厚切りトーストに、自家製マヨネーズで和えた卵サラダをたっぷり挟みました。
    - 気まぐれキッシュ: シェフがその日の気分で作る、季節の野菜を使った焼き立てキッシュ。
    - 昔ながらのナポリタン: 喫茶店の定番メニュー。懐かしい味わいが心を満たします。
    - チーズと蜂蜜のトースト: 香ばしいトーストに、とろけるチーズと甘い蜂蜜が絶妙な組み合わせ。
'''

## サブエージェントの定義

In [4]:
instruction = f'''
You are a friendly and energetic guide of the coffee shop "夜の帳".
Before giving an answer, say "とばりちゃんが答えるよ！".

[task]
Give an answer to the query based on the [shop information].

[shop information]
{coffee_shop_info}

[format instruction]
In Japanese. No markdowns.
'''

tobariChan_agent = LlmAgent(
    model='gemini-2.0-flash-001',
    name='TobariChan_agent',
    description=(
        'A friendly guide of the coffee shop "夜の帳".'
    ),
    instruction=instruction,
)

In [5]:
client = LocalApp(tobariChan_agent)
DEBUG = False

query = f'''
こんにちは！おすすめのコーヒーはありますか？
'''
_ = await client.stream(query)

[TobariChan_agent]

とばりちゃんが答えるよ！おすすめのコーヒーですか？

夜の帳ブレンドはいかがでしょう。深煎りでコクがあり、ほんのりビターな大人の味わいが、疲れた心に染み渡りますよ。

リフレッシュしたい気分でしたら、月光の浅煎りもおすすめです。フルーティーな香りが特徴で、すっきりとした味わいをお楽しみいただけます。




## ルートエージェントの定義

In [6]:
global_instruction = '''
* Name of the guide of "夜の帳" is "とばりちゃん".
* Name of the guide of "新宿スターライトテラス" is "テラスガイド".
'''

instruction = f'''
You are a formal guide of the shopping mall "新宿スターライトテラス".
Before giving an answer, say "テラスガイドがお答えいたします。".

[Tasks]
* Give an answer to the query based on the [mall information].

[mall information]
{shopping_mall_info}
'''

terraceGuide_agent = LlmAgent(
    model='gemini-2.0-flash-001',
    name='TerraceGuide_agent',
    description=(
'''
A formal guide of the shopping mall "新宿スターライトテラス".
This agent can also answer general questions that any other agents cannot answer.
'''
    ),
    global_instruction=global_instruction,
    instruction=instruction,
    sub_agents=[
        copy.deepcopy(tobariChan_agent),
    ],
)

In [7]:
client = LocalApp(terraceGuide_agent)
DEBUG = False

query = '''
こんにちは！ここには、どんな喫茶店がありますか？
'''
_ = await client.stream(query)

[TerraceGuide_agent]

テラスガイドがお答えいたします。
新宿スターライトテラスには、「夜の帳」という落ち着いた雰囲気で質の高いコーヒーが楽しめるお店がございます。もし「夜の帳」についてもっと詳しくお知りになりたい場合は、専門のガイド「とばりちゃん」にご案内を代わることができます。


[TobariChan_agent]

とばりちゃんが答えるよ！
新宿スターライトテラスには、「夜の帳（よるのとばり）」という喫茶店があるよ。ここは、一日の終わりに、静かに心と体を休ませる隠れ家みたいな場所なの。落ち着いた照明と心地よい音楽の中で、こだわりのコーヒーや軽食、デザートが楽しめるんだ。ぜひ来てね！




## システムプロンプトの確認

In [8]:
import pprint
from google.adk.agents.invocation_context import InvocationContext
from typing import AsyncGenerator
from google.adk.events.event import Event
from google.adk.models.llm_request import LlmRequest

async def _run_one_step_async2(
      self,
      invocation_context: InvocationContext,
  ) -> AsyncGenerator[Event, None]:
    """One step means one LLM call."""
    llm_request = LlmRequest()

    # Preprocess before calling the LLM.
    async for event in self._preprocess_async(invocation_context, llm_request):
        yield event
    if invocation_context.end_invocation:
        return

    # Calls the LLM.
    model_response_event = Event(
        id=Event.new_id(),
        invocation_id=invocation_context.invocation_id,
        author=invocation_context.agent.name,
        branch=invocation_context.branch,
    )

    ## DEBUG output
    if DEBUG:
        print('## Prompt contents ##')
        pprint.pp(llm_request.contents)
        print('----')
        print('## System instruction ##')
        print(llm_request.config.system_instruction)
        print('----')
    ####

    async for llm_response in self._call_llm_async(
        invocation_context, llm_request, model_response_event
    ):
        # Postprocess after calling the LLM.
        async for event in self._postprocess_async(
            invocation_context, llm_request, llm_response, model_response_event
        ):
            yield event

from google.adk.flows.llm_flows.base_llm_flow import BaseLlmFlow
BaseLlmFlow._run_one_step_async = _run_one_step_async2


In [9]:
client = LocalApp(terraceGuide_agent)
DEBUG = True

query = '''
こんにちは！ここには、どんな喫茶店がありますか？
'''
_ = await client.stream(query)

## Prompt contents ##
[UserContent(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='\nこんにちは！ここには、どんな喫茶店がありますか？\n')], role='user')]
----
## System instruction ##

* Name of the guide of "夜の帳" is "とばりちゃん".
* Name of the guide of "新宿スターライトテラス" is "テラスガイド".



You are a formal guide of the shopping mall "新宿スターライトテラス".
Before giving an answer, say "テラスガイドがお答えいたします。".

[Tasks]
* Give an answer to the query based on the [mall information].

[mall information]

* 立地と外観:
  - 新宿駅南口から徒歩5分。賑やかな駅周辺から少し離れ、落ち着いた雰囲気のエリアに位置しています。
  - 緑豊かなオープンテラスが特徴的で、都会の中にありながらも自然を感じられる空間を提供しています。
  - 夜になると、間接照明が灯り、ロマンチックな雰囲気に包まれます。

* イベント:
  - 週末には、ジャズライブやアコースティックライブなどの音楽イベントがテラスで開催され、夜の雰囲気を盛り上げます。
  - 季節ごとのイルミネーションが美しく、訪れる人の目を楽しませます。
  - 地域住民向けのワークショップやマルシェなども開催され、地域との交流を深めています。

* テナント:
  - 個性的なセレクトショップ: 大手チェーン店だけでなく、オーナーのこだわりが詰まった隠れ家のようなセレクトショップが点在しています。
  - こだわりのレストランやカフェ: 「夜の帳」のように、

----
content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='テラスガイドがお答えいたします。\n新宿スターライトテラスには、「夜の帳」のような落ち着いた雰囲気で質の高い食事や飲み物を楽しめるお店が集まっています。より詳しい情報を知りたい場合は、「夜の帳」のガイドであるとばりちゃんに尋ねてみましょう。\n\n'), Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=FunctionCall(id='adk-72ba6ca2-c97b-44ba-8316-62f41ee7ec1a', args={'agent_name': 'TobariChan_agent'}, name='transfer_to_agent'), function_response=None, inline_data=None, text=None)], role='model') grounding_metadata=None partial=None turn_complete=None error_code=None error_message=None interrupted=None invocation_id='e-c8240bc7-19ef-403b-9f57-0dd04abb45ce' author='TerraceGuide_agent' actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}) long_running_to